In [4]:
# # 사용할 모듈 정의
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import models, layers,Input
import sys,os,math,pathlib
from tensorflow.keras.models import Sequential
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
import time

# 새로 학습할 데이터가 있는 디렉토리
new_data_dir='C:/Users/jaehee/Desktop/graduation_project/voice_data_files/2021_9/09_13/jaehee(old)_denoise(per5)&(new)ori+aug(30_morning)/bad'
batch_size = 32
split_percent=0.3
rand_seed=123
shuffle_num=1000
EPOCHS=150
# 모델 세이브
save_model_date="09_13"
save_model_name="bad(old(ori(5)&new(sangho_ori+aug(30))E(150)layers(_-1)))"
save_model_dir='C:/Users/jaehee/Desktop/jupyter_proj/model'

# .png 형식의 데이터 인식
data_dir = pathlib.Path(new_data_dir)
image_list = list(data_dir.glob('*/*.png'))
image_size = Image.open(image_list[0]).size
img_width=image_size[0]
img_height=image_size[1]

# train data set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split_percent,
    subset="training",
    seed=rand_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# test data set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split_percent ,
    subset="validation",
    seed=rand_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# data preprocessing
class_names = train_ds.class_names
num_classes = len(class_names)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(shuffle_num).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))


# 모델 불러오기
# 기본 폴더 + 사용자 이메일 + 모델 이름 --> 이렇게 불러와야 할 듯
    # 사용자 별로 모델이 다를 것이기 때문에 템플릿 복사시 모델도 복사를 해주자

load_model_name="aug(bad)+origin(bad)(batch=32).h5"
load_model_date="09_08"
load_model_dir='C:/Users/jaehee/Desktop/jupyter_proj/model'

test_h5 = load_model(load_model_dir+'/'+load_model_date+'/'+load_model_name)

# dense 층 제외 가중치 잠그기
for layer in test_h5.layers[:-1]:
    layer.trainable = False
    
#클래스 추가 없는 레이어
maintain_class_layer = test_h5.layers[-1]

#클래스 추가가 있는 레이어
diff = test_h5.layers[-1].get_weights()[1].shape[0] - num_classes
if (diff!=0):
    add_class_layer = test_h5.layers[-2].output
    add_class_layer = Dense(num_classes, activation='softmax')(add_class_layer)
    transfer_model = Model(test_h5.input, outputs=add_class_layer)
    
    # 이전 모델의 dense layer weights를 backup
    weights_bak = test_h5.layers[-1].get_weights()
    # 새로운 모델의 dense layer weights를 backup
    weights_new = transfer_model.layers[-1].get_weights()
    # 새로운 모델의 dense layer weights에 이전 모델의 dense layer weights를 이전
    weights_new[0][:, :diff] = weights_bak[0]
    weights_new[1][:diff] = weights_bak[1]
    transfer_model.layers[-1].set_weights(weights_new)
    
else:
    transfer_model = Model(test_h5.input, maintain_class_layer(test_h5.layers[-2].output))

transfer_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#start = time.time()
history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)
#time= time.time() - start


#h5 타입
transfer_model.save(save_model_dir+'/'+save_model_date+'/'+save_model_name+'.h5')
#tf 타입
transfer_model.save(save_model_dir+'/'+save_model_date+'/'+save_model_name, save_format='tf')



Found 60 files belonging to 7 classes.
Using 42 files for training.
Found 60 files belonging to 7 classes.
Using 18 files for validation.
Epoch 1/150
2/2 [==============================] - 3s 990ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/150
2/2 [==============================] - 0s 205ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/150
2/2 [==============================] - 0s 144ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/150
2/2 [==============================] - 0s 142ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/150
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 